In [2]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
import pytesseract
import easyocr
import math
from paddleocr import PaddleOCR



# def ocr_detect(reader, idx, x_center, y_center, radius, ocr_regions_folder, image):
def ocr_detect(reader, ocr, idx, x_center, y_center, radius, ocr_regions_folder, image, image_path):

    # (x_center, y_center, radius)=circular_region

    # Ensure coordinates and radius are integers and within bounds
    x_center, y_center, radius = int(x_center), int(y_center), int(radius)
    # x_center = max(0, min(x_center, image.shape[0] - 1))
    # y_center = max(0, min(y_center, image.shape[1] - 1))
    # radius = max(1, min(radius, min(image.shape[1], image.shape[0]) // 2))

    # Create a black mask with a white-filled circle
    mask = np.zeros_like(image, dtype=np.uint8)
    cv2.circle(mask, (x_center, y_center), radius, (255, 255, 255), thickness=-1)

    # Apply the mask to extract the circular region
    circular_region = cv2.bitwise_and(image, image, mask=mask[:, :, 0])
    denoised = cv2.GaussianBlur(circular_region, (3,3), 0)

    # Convert to grayscale explicitly before equalization
    # if len(denoised.shape) == 3:
    #     gray = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)

    # laplacian = cv2.Laplacian(circular_region, cv2.CV_64F)
    # sharp = cv2.convertScaleAbs(laplacian)
    # cv2.imwrite("sharp_image.jpg", sharp)

    # equalized = cv2.equalizeHist(gray)
    # cv2.imwrite("equalized_image.jpg", equalized)

    # denoised = equalized
    

    # Save or visualize the edge-detected circular region
    denoised_path = os.path.join(ocr_regions_folder, f"denoised_circle_{idx}_{os.path.basename(image_path)}")
    cv2.imwrite(denoised_path, circular_region)
    print(f"Saved denoised circular region {idx} to: {denoised_path}")

    # # Apply Canny edge detection
    # edges = cv2.Canny(circular_region, 100, 300)

    # Save or visualize the edge-detected circular region
    # edges_path = os.path.join(ocr_regions_folder, f"edges_circle_{idx}_{os.path.basename(image_path)}")
    # cv2.imwrite(edges_path, circular_region)
    # print(f"Saved edge-detected circular region {idx} to: {edges_path}")

    # # OCR on the edge-detected region
    # ocr_text = pytesseract.image_to_string(denoised, config='--psm 6')  # Use PSM 6 for a single block of text
    # # print(f"OCR result for circular region {idx}: {ocr_text}")

    
    
    # # Optionally, save the OCR result
    # with open(os.path.join(output_folder, f"ocr_circle_{idx}_{os.path.basename(image_path)}.txt"), "w") as f:
    #     f.write(ocr_text)
    

    result = ocr.ocr(denoised, cls=True)
    # detected_number = None

    # Extract the detected text (integer)
    if result and result[0]:  # Ensure result is not empty
        # detected_text = result[0][0][1][0]  # Get the recognized text
        try:
            # detected_number = int(detected_text)  # Convert to integer
            return float(result[0][0][1][0])
        except ValueError:
            # detected_number = None  # Handle cases where conversion fails (e.g., text is non-numeric)
            print(f"PaddleOCR could not convert the text to integer: {result[0][0][1][0]}")

        # print(detected_number)  # Output the extracted integer

    value=""
    results = reader.readtext(denoised)
    for (bbox, text, confidence) in results:
        # print(f"Detected text: {text} (Confidence: {confidence:.2f})")
        value+=text
    
    # value=ocr_text.strip()
    # Try to convert the detected text to an integer
    try:
        return float(value)  # Attempt to convert the text to an integer
    except ValueError:
        print(f"EasyOCR could not convert the text to integer: {value}")
        # return -2



    

    return None

# calculate clockwise angle from negative y axis
# Adjust angle calculation for elliptical gauge distortion
def rotate_point(point, angle_of_inclination):
    # Apply rotation matrix to the point
    rotation_matrix = np.array([
        [math.cos(angle_of_inclination), -math.sin(angle_of_inclination)],
        [math.sin(angle_of_inclination), math.cos(angle_of_inclination)]
    ])
    rotated_point = np.dot(rotation_matrix, point)
    return rotated_point

def stretch_point(point, major_axis_length, minor_axis_length):
    # Stretch the point based on ellipse's axes
    stretch_factor = major_axis_length / minor_axis_length
    stretched_point = np.array([point[0] * stretch_factor, point[1]])
    return stretched_point

# def get_angle(point1, point2, point3, major_axis_length, minor_axis_length, ellipse_inclination):
def get_angle(point1, point2, major_axis_length, minor_axis_length, ellipse_inclination):
    """
    Compute the counterclockwise angle (in radians) from the positive x-axis between two points,
    then apply an elliptical correction factor based on the ratio of major_axis_length to minor_axis_length.
    
    Parameters:
      point1, point2: tuples (x, y) representing the two points.
      major_axis_length: the length of the ellipse's major axis.
      minor_axis_length: the length of the ellipse's minor axis.
    
    Returns:
      corrected_angle: The corrected angle (in radians) measured counterclockwise from the positive x-axis.
    """
    if point1 is None or point2 is None:
        return None  # Return None if any point is missing

    # Rotate the points based on the ellipse's inclination
    rotated_point1 = rotate_point(point1, ellipse_inclination)
    rotated_point2 = rotate_point(point2, ellipse_inclination)
    
    # Stretch the points based on ellipse axes (accounting for elliptical distortion)
    stretched_point1 = stretch_point(rotated_point1, major_axis_length, minor_axis_length)
    stretched_point2 = stretch_point(rotated_point2, major_axis_length, minor_axis_length)

    x1, y1 = point1
    x2, y2 = point2

    # Calculate the raw angle from the positive x-axis (counterclockwise), normalized to [0, 2*pi].
    raw_angle = (math.atan2(y2 - y1, x2 - x1) + 2 * math.pi) % (2 * math.pi)
    
    # Compute the elliptical correction factor.
    ellipse_correction_factor = major_axis_length / minor_axis_length
    
    # Apply the correction factor.
    corrected_angle = raw_angle * ellipse_correction_factor
    
    # Optionally, normalize the corrected angle to the range [0, 2*pi].
    corrected_angle = corrected_angle % (2 * math.pi)

    # while corrected_angle < 0:
    #     corrected_angle += 2 * math.pi

    # while corrected_angle >= 2 * math.pi:
    #     corrected_angle -= 2 * math.pi
    
    return corrected_angle


def get_values(max_angle, min_angle, pointer_angle):
    """
    Given:
      - min_angle: the angle (in radians) of the minimum reading.
      - max_angle: the angle (in radians) of the maximum reading.
      - pointer_angle: the angle (in radians) of the needle pointer.
    
    Assumes that, in the "correct" gauge orientation, the minimum lies to the left and the maximum to the right.
    
    There are four general cases (with some duplicates in the original logic):
    
    (i)  If min_angle < max_angle < pointer_angle:
         The obtuse sector has the scale and the needle moves in clockwise direction,
         with the reference line lying in the scaled sector and the pointer below the reference.
         => angle_range = 2π - (max_angle - min_angle)
         => val_angle   = 2π - (pointer_angle - min_angle)
    
    (ii) If pointer_angle < min_angle < max_angle:
         The obtuse sector has the scale and the needle moves in clockwise direction,
         with the reference line in the scaled sector and the pointer above the reference.
         => angle_range = 2π - (max_angle - min_angle)
         => val_angle   = min_angle - pointer_angle
         
    (iii) If max_angle < pointer_angle < min_angle:
         The obtuse sector has the scale, the reference line lies outside the scaled sector,
         and the needle moves in clockwise direction.
         => angle_range = min_angle - max_angle
         => val_angle   = min_angle - pointer_angle
         
    (iv) If min_angle < pointer_angle < max_angle:
         The acute sector has the scale, the reference line lies outside the scaled sector,
         and the needle moves in counterclockwise direction.
         => angle_range = max_angle - min_angle
         => val_angle   = pointer_angle - min_angle
         
    (v)  If pointer_angle < max_angle < min_angle:
         (This case covers the situation where the acute sector is used, the reference line is in the scaled sector,
         and the needle moves in counterclockwise direction with the pointer above (or below) the reference.)
         Here we adjust pointer_angle:
         => angle_range = 2π - (min_angle - max_angle)
         => pointer_angle_adj = 2π - (min_angle - pointer_angle)
         => val_angle = pointer_angle_adj

    Returns:
      angle_range: the effective angular range of the scale (in radians)
      val_angle: the pointer's relative angle (in radians) within that range.
    """
    if min_angle < max_angle < pointer_angle:
        # Case (i)
        angle_range = 2 * math.pi - (max_angle - min_angle)
        val_angle = 2 * math.pi - (pointer_angle - min_angle)
        print(f'case (i) : ')
    elif pointer_angle < min_angle < max_angle:
        # Case (ii) -- covers duplicate branch
        angle_range = 2 * math.pi - (max_angle - min_angle)
        val_angle = min_angle - pointer_angle
        print(f'case (ii) : ')
    elif max_angle < pointer_angle < min_angle:
        # Case (iii)
        angle_range = min_angle - max_angle
        val_angle = min_angle - pointer_angle
        print(f'case (iii) : ')
    elif min_angle < pointer_angle < max_angle:
        # Case (iv)
        angle_range = max_angle - min_angle
        val_angle = pointer_angle - min_angle
        print(f'case (iv) : ')
    elif pointer_angle < max_angle < min_angle:
        # Case (v)
        angle_range = 2 * math.pi - (min_angle - max_angle)
        val_angle = 2 * math.pi - (min_angle - pointer_angle)
        print(f'case (v) : ')
    elif max_angle < min_angle < pointer_angle:
        # Case (vi)
        angle_range = 2 * math.pi - (min_angle - max_angle)
        val_angle = pointer_angle - min_angle
        print(f'case (vi) : ')
    else:
        # If none of the above conditions hold, return None or raise an error.
        angle_range = None
        val_angle = None
        print(f'case (vii) : ')
    
    print(f'angle_range = {math.degrees(angle_range)}')
    print(f'val_angle = {math.degrees(val_angle)}')
    
    return angle_range, val_angle


if __name__ == '__main__':

    segmentation_model = YOLO("yolo11n-seg.pt")
    detection_model = YOLO("C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\pretrained_models\\5\\last.pt")
    
    # Specify the folder containing images
    test_images_folder = "C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6"
    # masked_images_folder = "C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\masked_images"
    # os.makedirs(masked_images_folder, exist_ok=True)

    output_folder = "C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\result_images_6"
    os.makedirs(output_folder, exist_ok=True)

    ocr_regions_folder = "C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\ocr_regions_6"
    os.makedirs(ocr_regions_folder, exist_ok=True)

    # # Dictionary to store ellipse parameters
    # ellipse_parameters = {}

    # Get a list of all image file paths in the folder
    image_paths = [os.path.join(test_images_folder, img) for img in os.listdir(test_images_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Run the YOLO segmentation model on the images
    # segmentation_results = segmentation_model(image_paths)


    # Define the path to the image and Tesseract executable (adjust if needed)
    # pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\Harsh\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
    

    # Initialize the Reader for English (or multiple languages)
    reader = easyocr.Reader(['en'], gpu=True)

    # # Initialize OCR with custom model path
    # ocr = PaddleOCR(
    #     det_model_dir="en_PP-OCRv3_det_distill_train",
    #     use_angle_cls=True,
    #     lang="en"
    # )

    # Initialize PaddleOCR with CPU
    ocr = PaddleOCR(use_angle_cls=True, lang='en', rec=True, device='cpu')

    results = None

    good_images=set()
    good_images_val={}

    for image_path in image_paths:
        print(f"Processing results for: {image_path}")

        # Open the image
        image = cv2.imread(image_path)

        # Get image dimensions (height, width, channels)
        height, width, channels = image.shape
        
        major_axis_length = None
        minor_axis_length = None

        # Run the YOLO segmentation model on the opened image
        segmentation_results = segmentation_model(image_path)

        # Ensure segmentation results exist for the image
        if segmentation_results[0].masks is not None:
            # Get masks and class IDs
            # All masks
            masks = segmentation_results[0].masks.data.cpu().numpy()
            # Class IDs for the masks
            class_ids = segmentation_results[0].boxes.cls.cpu().numpy()


            # Check for the 'clock' class mask (replace 74 with the correct class ID for 'clock')
            clock_class_id = 74
            clock_mask = None
            for i in range(len(class_ids)):
                class_id = class_ids[i]
                if int(class_id) == clock_class_id:
                    clock_mask = masks[i]
                    break

            if clock_mask is not None:
                # Process the clock mask
                clock_mask = (clock_mask * 255).astype(np.uint8)

                # Resize mask to match the original image size
                height, width = image.shape[:2]
                mask_resized = cv2.resize(clock_mask, (width, height), interpolation=cv2.INTER_LINEAR)

                # Clean the mask using morphological operations
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
                cleaned_mask = cv2.morphologyEx(mask_resized, cv2.MORPH_CLOSE, kernel)
                cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel)

                # Find contours in the cleaned mask
                contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                valid_contours = [c for c in contours if cv2.contourArea(c) > 500]

                if valid_contours:
                    # Get the largest contour
                    largest_contour = max(valid_contours, key=cv2.contourArea)

                    # Fit an ellipse to the largest contour
                    ellipse = cv2.fitEllipse(largest_contour)

                    # Extract major and minor axes and angle
                    center, axes, angle = ellipse
                    major_axis_length = max(axes)
                    minor_axis_length = min(axes)

                    # # Save ellipse parameters
                    # ellipse_parameters[os.path.basename(image_path)] = {
                    #     "major_axis_length": major_axis_length,
                    #     "minor_axis_length": minor_axis_length,
                    #     "angle": angle
                    # }

                    # Draw the fitted ellipse on the original image
                    segmented_image = np.zeros_like(image)  # Black background
                    # Apply mask
                    segmented_image[cleaned_mask > 0] = image[cleaned_mask > 0]
                    

                    cv2.ellipse(segmented_image, ellipse, (0, 255, 0), 2)
                    
                    # cv2.circle(segmented_image, (int(center[0]), int(center[1])), radius=5, color=(0, 255, 0), thickness=-1)
                    
                    
                    # # Print or save ellipse parameters
                    # print("Ellipse Parameters:", ellipse_parameters)
                    # with open(os.path.join(masked_images_folder, "ellipse_parameters.txt"), "w") as f:
                    #     for image_name, params in ellipse_parameters.items():
                    #         f.write(f"{image_name}: {params}\n")


                    # # Save the segmented image
                    # output_path = os.path.join(masked_images_folder, f"segmented_{os.path.basename(image_path)}")
                    # cv2.imwrite(output_path, segmented_image)
                    # print(f"Processed and saved to: {output_path}")

                    results = detection_model(segmented_image)
                else:
                    print(f"No valid contours found for 'clock' in: {image_path}")
                    results = detection_model(segmented_image)
            else:
                print(f"No 'clock' mask found in: {image_path}")
                results = detection_model(image)
        else:
            print(f"No segmentation results for: {image_path}")
            results = detection_model(image)
        
        
        # print(f"Processing results for: {image_paths[index]}")
        

        if results is None:
            continue

        for result in results:
            # Extract bounding boxes and confidence scores
            # Bounding boxes (x_min, y_min, x_max, y_max)
            boxes = result.boxes.xyxy.cpu().numpy()
            # Confidence scores
            confs = result.boxes.conf.cpu().numpy()
            # Class labels
            labels = result.boxes.cls.cpu().numpy()

            # Dictionary to store the highest-confidence box center for each label
            label_centers = {}

            # for i, label in enumerate(labels):
            for i in range(len(labels)):
                label=int(labels[i])
                # x-center of the box
                center_x = (boxes[i][0] + boxes[i][2]) / 2
                # y-center of the box
                center_y = (boxes[i][1] + boxes[i][3]) / 2
                confidence = confs[i]

                # Update the label's center if it has a higher confidence score
                if (label not in label_centers) or (confidence > label_centers[label]['confidence']):
                    label_centers[label] = {'center': (center_x, center_y), 'confidence': confidence}

            # Print or save the centers for this image
            print(f"Label centers for {image_path}:")
            for label in label_centers:
                data = label_centers[label]
                print(f"  Label: {label}, Center: {data['center']}, Confidence: {data['confidence']}")

            # Optionally save the results visually with points drawn
            # output_folder = "results"  # Specify your desired output directory
            # os.makedirs(output_folder, exist_ok=True)


            # Visualize centers on the image
            # Use YOLO's built-in plotting
            result_plotted = result.plot(labels=True)

            

            for data in label_centers.values():
                center_x, center_y = data['center']
                # center_y = height - center_y
                # data['center'][1] = height - data['center'][1]

                # Draw the center as a small circle on the image (modify result_plotted)
                cv2.circle(result_plotted, (int(center_x), int(center_y)), radius=5, color=(0, 255, 0), thickness=-1)

            # cv2.ellipse(result_plotted, ellipse, (0, 255, 0), 2)

            # label_names=["base", "maximum", "minimum", "tip"]
            
            # Safely extract the center coordinates for each label
            # Label 0
            pointer_base_1 = label_centers.get(0, {}).get('center', None)
            # Label 1
            maximum_1 = label_centers.get(1, {}).get('center', None)
            # Label 2
            minimum_1 = label_centers.get(2, {}).get('center', None)
            # Label 3   
            pointer_tip_1 = label_centers.get(3, {}).get('center', None)

            # # Get image dimensions (height, width, channels)
            # height, width, channels = image.shape

            # pointer_base[1] = height - pointer_base[1]
            # maximum[1] = height - maximum[1]
            # minimum[1] = height - minimum[1]
            # pointer_tip[1] = height - pointer_tip[1]
            
            pointer_base = None
            pointer_tip = None
            maximum = None
            minimum = None


            if pointer_base_1 is not None :
                pointer_base = [pointer_base_1[0], height - pointer_base_1[1]]
            else :
                continue

            if pointer_tip_1 is not None :
                pointer_tip = [pointer_tip_1[0], height - pointer_tip_1[1]]
            else :
                continue

            if maximum_1 is not None :
                maximum = [maximum_1[0], height - maximum_1[1]]
            else :
                continue

            if minimum_1 is not None :
                minimum = [minimum_1[0], height - minimum_1[1]]
            else :
                continue


            # Print the extracted values
            print(f"Pointer Base: {pointer_base}")
            print(f"Maximum: {maximum}")
            print(f"Minimum: {minimum}")
            print(f"Pointer Tip: {pointer_tip}")

            
            # circular_regions = {}
            # circular_regions['max']=[]
            # circular_regions['min']=[]
            
            needle_length = math.sqrt(((pointer_base[0] - pointer_tip[0]) ** 2) + ((pointer_base[1] - pointer_tip[1]) ** 2))

            if major_axis_length is None:
                major_axis_length = 3 * needle_length
            
            if minor_axis_length is None:
                minor_axis_length = 3 * needle_length
            
            if minimum_1 is not None:
                cv2.circle(result_plotted, (int(minimum_1[0]), int(minimum_1[1])), radius=int(major_axis_length/10), color=(0, 0, 255), thickness=2)
                # circular_regions['min'].append([minimum[0], minimum[1], major_axis_length/10])
                min_val=ocr_detect(reader, ocr, 0, minimum_1[0], minimum_1[1], major_axis_length/9, ocr_regions_folder, image, image_path)
                print(f"Minimum Value = {min_val}")
                
            
            if maximum_1 is not None:
                cv2.circle(result_plotted, (int(maximum_1[0]), int(maximum_1[1])), radius=int(major_axis_length/10), color=(0, 0, 255), thickness=2)
                # circular_regions['max'].append([maximum[0], maximum[1], major_axis_length/10])
                max_val=ocr_detect(reader, ocr, 1, maximum_1[0], maximum_1[1], major_axis_length/9, ocr_regions_folder, image, image_path)
                print(f"Maximum Value = {max_val}")


                # try:
                #     max_val=ocr_detect(reader, 1, circular_regions[1], ocr_regions_folder, image)
                #     print(f"Maximum Value = {max_val}")
                # except IndexError:
                #     max_val=ocr_detect(reader, 0, circular_regions[0], ocr_regions_folder, image)
                #     print(f"Maximum Value = {max_val}")
            


            output_path = os.path.join(output_folder, f"result_{os.path.basename(image_path)}")
            # Save the transformed image
            cv2.imwrite(output_path, result_plotted)
            print(f"Results saved to: {output_path}")
            
            ellipse_inclination = angle

            final_value=None

            # Example processing of the gauge value
            if (minimum is not None) and \
                (maximum is not None) and \
                    (pointer_base is not None) and \
                        (pointer_tip is not None):
                
                
                # Get the angles for minimum, maximum, and pointer positions
                min_angle = get_angle(pointer_base, minimum, major_axis_length, minor_axis_length, ellipse_inclination)
                max_angle = get_angle(pointer_base, maximum, major_axis_length, minor_axis_length, ellipse_inclination)
                pointer_angle = get_angle(pointer_base, pointer_tip, major_axis_length, minor_axis_length, ellipse_inclination)
                
                
                # If all the angles are available, perform linear interpolation with corrected angles
                if (max_val is not None) and \
                    (min_val is not None) and \
                    (max_angle is not None) and \
                    (min_angle is not None) and \
                    (pointer_angle is not None):
                    
                    print(f'min_angle = {math.degrees(min_angle)}')
                    print(f'max_angle = {math.degrees(max_angle)}')
                    print(f'pointer_angle = {math.degrees(pointer_angle)}')
                    
                    angle_range, val_angle = get_values(max_angle, min_angle, pointer_angle)
                    
                    if (angle_range is not None) and\
                        (val_angle is not None):
                        final_value = ((max_val - min_val) * val_angle) / angle_range
                        
                        good_images.add(image_path)
                        good_images_val[image_path] = final_value
                        print(f'final_value = {final_value}')

            

            # angle_range=angle_between(pointer_base, minimum, maximum, major_axis_length, minor_axis_length, angle)
            # val_angle=angle_range=angle_between(pointer_base, minimum, pointer_tip, major_axis_length, minor_axis_length, angle)
            # left_angle=angle_range=angle_between(pointer_base, pointer_tip, maximum, major_axis_length, minor_axis_length, angle)

            # if (angle_range is not None) and \
            #     (val_angle is not None) and \
            #     (left_angle is not None) and \
            #     (max_val is not None) and \
            #         (min_val is not None):
                
            #     print(f'max_val - min_val = {max_val-min_val}')
            #     print(f'max_angle-min_angle = {angle_range}')
            #     print(f'pointer_angle-min_angle = {val_angle}')
            #     print(f'pointer_angle-max_angle = {left_angle}')

            #     epsilon = 1e-4  # Small tolerance for floating point comparison

            #     # Check if angle_range + val_angle is approximately equal to left_angle
            #     if abs((angle_range + val_angle) - left_angle) < epsilon:
            #         angle_range = 2 * math.pi - angle_range
            #     # elif(angle_range==val_angle+left_angle):
            #     #     continue

            #     print('after checking')
            #     print(f'max_val - min_val = {max_val-min_val}')
            #     print(f'max_angle - min_angle = {angle_range}')
            #     print(f'pointer_angle-min_angle = {val_angle}')
            #     print(f'pointer_angle - max_angle = {left_angle}')

            #     final_value=final_value = ((max_val - min_val) * val_angle) / angle_range
            
            #     good_images.add(image_path)
            #     good_images_val[image_path] = final_value
            #     print(final_value)


        print("Processing completed.")

    original_values = {}
    least_counts = {}


    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image1.jpg"]=125
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image1.jpg"]=10

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image2.jpg"]=8.875
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image2.jpg"]=0.2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image4.jpg"]=0
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image4.jpg"]=0.1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image8.jpg"]=49.5
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image8.jpg"]=1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image10.jpg"]=0.625
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image10.jpg"]=0.05

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image11.jpg"]=10
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image11.jpg"]=2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image13.jpg"]=-0.95
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image13.jpg"]=0.1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image21.jpg"]=1.6
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image21.jpg"]=0.2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image22.jpg"]=1.65
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image22.jpg"]=0.2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image29.jpg"]=80
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image29.jpg"]=5

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image30.jpg"]=0.57
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image30.jpg"]=0.2
    
    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image32.jpg"]=8.875
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image32.jpg"]=0.2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image35.jpg"]=-5
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image35.jpg"]=1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image42.jpg"]=0
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image42.jpg"]=0.1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image43.jpg"]=0
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image43.jpg"]=5

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image44.jpg"]=18
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image44.jpg"]=2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image46.jpg"]=18
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image46.jpg"]=2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image48.jpg"]=20
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image48.jpg"]=5

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image49.jpg"]=-6
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image49.jpg"]=2

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image51.jpg"]=4.8
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image51.jpg"]=0.1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image53.jpg"]=6.25
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image53.jpg"]=0.5

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image54.jpg"]=4.8
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image54.jpg"]=0.1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image57.jpg"]=92
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image57.jpg"]=5

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image62.jpg"]=26.5
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image62.jpg"]=0.2
    
    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image68.jpg"]=8.82
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image68.jpg"]=0.2
    
    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image72.jpg"]=27.25
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image72.jpg"]=1

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image74.jpg"]=1.1
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image74.jpg"]=0.05

    original_values["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image75.png"]=39
    least_counts["C:\\Users\\Harsh\\Desktop\\PTZ Camera Zoom with Gauge Detection\\test_images_6\\image75.png"]=2
    

    err_sum=0
    errlc_sum=0
    count=0
    mse_sum=0
    abs_sum=0
    ss_res_sum=0
    ss_tot_sum=0
    act_val_sum=0
    pred_val_sum=0
    error = 0
    
    # Print final results for good images
    for good_image_path in good_images:
        abs_error = abs(good_images_val[good_image_path] - original_values[good_image_path])

        if original_values[good_image_path] > 0 :
            error = (100 * abs_error) / abs(original_values[good_image_path])

        err_sum += error
        # error_per_least_count = (100 * abs(original_values[good_image_path] - good_images_val[good_image_path])) / (original_values[good_image_path] * least_counts[good_image_path])
        # errlc_sum += error_per_least_count
        mse_sum += (good_images_val[good_image_path] - original_values[good_image_path]) ** 2
        abs_sum += abs_error

        act_val_sum += original_values[good_image_path]
        pred_val_sum += good_images_val[good_image_path]

        count += 1

        print(f"{good_image_path} :- ")
        print(f"Detected Value = {good_images_val[good_image_path]}")
        print(f"Original Value = {original_values[good_image_path]}")
        print(f"% Error = {error}")
        print(f"Absolute Error = {abs_error}")
        print(f"\n")

    print(f"Number of images processed = {count}")

    if count > 0 :
        print(f"Mean % Error = {err_sum/count}")
        print(f"Mean Absolute Error = {abs_error/count}")
        print(f"Root Mean Square Error = {math.sqrt(mse_sum)/count}")

    # Print final results for good images
    for good_image_path in good_images:
        ss_res_sum += (original_values[good_image_path] - good_images_val[good_image_path]) ** 2
        if count > 0 :
            ss_tot_sum += (original_values[good_image_path] - (act_val_sum / count)) ** 2
    
    if ss_tot_sum > 0 :
        print(f"R_square = {1-(ss_res_sum/ss_tot_sum)}")



[2025/08/25 22:56:49] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Harsh/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Harsh/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=